## Autocorrelation Functions for Restricted Diffusion

Python Jupyter notebook for calculations performed in:

Breaking Down Walls: Continuous Potential Models for Internal Motions in NMR Spin Relaxation

Arthur G. Palmer, III

J. Magn. Reson. in press (2024)

In [ ]:
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib import rcParams
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import HTML
from scipy.optimize import curve_fit
from scipy.linalg import expm as expm
from scipy.special import iv as iv
from scipy.special import lpmv as lpmv
from scipy.special import erf as erf
from scipy.special import erfi as erfi

%matplotlib widget

In [ ]:
# Colors for color blindness

ReddishPurple = [204/255.0,121/255.0,167/255.0]
Orange = [230/255.0,159/255.0,0/255.0]
Green = [0,158/255.0,115/255.0]
Blue = [0,114/255.0,178/255.0]
SkyBlue = [86/255.0,180/255.0,233/255.0]
Vermillion = [213/255.0,95/255.0,0.0]

In [ ]:
# Calculate ACF using FFT

def ACFfft(q,corrstride=1):
    """Autocorrelation function of input trajectory.
    Unbiased autorcorrelation function by FFT
    corrdim         length of correlation function (< len(q)/2)
    corrstride      points to skip to get to longer times
    Best performance if len(q) is a power of 2
    """
    
    f = q[::corrstride]
    
    N = f.shape[0]

    fvi = np.fft.fft(f, n=2*N)

    acf = np.real( np.fft.ifft( fvi * np.conjugate(fvi) )[:N] )

    acf = acf/(N-np.arange(N))
    
    acf =acf -np.mean(f)*np.mean(np.conjugate(f))
    
    return np.real(acf)

In [ ]:
# functions for existing simple models

# Diffusion-in-a-cone with rigid boundary

def DiffConeS2(thetacone0):
    thetacone = thetacone0*np.pi/180.0
    x0 = np.cos(thetacone)
    S2 = ((1/2)*x0*(1+x0))**2
    return S2

def DiffConete(thetacone0,tauc):
    
    S2 = DiffConeS2(thetacone0)

    thetacone = thetacone0*np.pi/180.0
    x0 = np.cos(thetacone)
    
    tmp1 = x0**2*(1+x0)**2*(np.log((1+x0)/2)+(1-x0)/2)/(2*(x0-1)) 
    tmp2 = (1/24.0)*(1-x0)*(6+8*x0-x0**2-12*x0**3-7*x0**4)
    te = 6*tauc*(tmp1 + tmp2)
    te = te/(1-S2)
    return te

def DiffConeACF(tauaxis,thetacone0,tauc):
    S2 = DiffConeS2(thetacone0)
    te = DiffConete(thetacone0,tauc)
    ACF =np.exp(-tauaxis/te)*(1.0-S2)+S2
    return ACF

# average value of theta for diffusion-in-a-cone
def avethetadcone(theta0):
    temp = (-theta0*np.cos(theta0)+np.sin(theta0))/(1-np.cos(theta0))
    return temp

# S2 for Bruschweiler GAF model
def GAF(thetaD,varphi):
    thetaR = thetaD*np.pi/180.0
    S2 = 1-3*np.sin(thetaR)**2*(np.cos(thetaR)**2*(1-np.exp(-varphi)) + (1/4.0)*np.sin(thetaR)**2*(1-np.exp(-4*varphi)))
    return S2

# Extended Lipari-Szabo model ACF
def extLS(taxis,Sf2,S2,tauf,taus):
        eLS = S2 + (1-Sf2)*np.exp(-taxis/tauf) + (Sf2-S2)*np.exp(-taxis/taus)
        return eLS
    

## von Mises hindered diffusion on cone

In [ ]:
def vonMisesPdist(thetaD,e0):
    thetaR = thetaD*np.pi/180
    tmp = np.exp((e0/2)*np.cos(thetaR))/(2*np.pi*iv(0,e0/2))
    return np.sqrt(tmp)

def BesselRatio(qq,e0):
    if e0 < 300:
        r = (iv(abs(qq),e0/2)/iv(0,e0/2))
    else:
        f0 = 1 + 1/(4*e0) + 9/(2*(4*e0)**2) + 75/(2*(4*e0)**3)
        u = 4*abs(qq)**2
        fqq = 1 - (u-1)/(4*e0) + (u-1)*(u-2)/(2*(4*e0)**2) - (u-1)*(u-2)*(u-3)/(6*(4*e0)**3)
        r = fqq/f0
    return r

def vonMisesEntropy(e0):
    if e0 < 300:
        vment = np.log(2*np.pi*iv(0,e0/2)) - (e0/2)*iv(1,e0/2)/iv(0,e0/2)
    else:
        tmp =(e0/2) - np.log(np.pi*e0)/2 + np.log(1 + 1/(4*e0) + 9/(2*(4*e0)**2) + 75/(2*(4*e0)**3))
        vment = tmp - (e0/2)*BesselRatio(1,e0)
    return vment

def vonMisesS2(thetaD,e0):
    thetaR = thetaD*np.pi/180
    q = np.array([-2,-1,0,1,2])
    y20 = (3*np.cos(thetaR)**2-1)/2
    y21 = -np.sqrt(3.0/2.0)*np.sin(thetaR)*np.cos(thetaR)
    y22 = np.sqrt(3.0/8.0)*np.sin(thetaR)**2
    Yarray = np.array([y20,y21,y22,y22,-y21])
    
    S2mises = 0
    for qq in range(-2,3) :
        S2mises = S2mises + BesselRatio(abs(qq),e0)**2*Yarray[qq]**2
    return(S2mises)

def vonMisesACF(timeax,thetaD,e0,Mdim=20):
    # timeax is unitless = Dt

    thetaR = thetaD*np.pi/180.0
    y20 = (3*np.cos(thetaR)**2-1)/2
    y21 = np.sqrt(3.0/2.0)*np.sin(thetaR)*np.cos(thetaR)
    y22 = np.sqrt(3.0/8.0)*np.sin(thetaR)**2
    yarray = np.array([y20**2,y21**2,y22**2,y22**2,y21**2])
    
    S2 = vonMisesS2(thetaD,e0)

    # M matrix should have even dimensions
    matdim = Mdim + 1
    if matdim % 2 == 0:
        matdim = matdim+1
        
    ntime = len(timeax)
    rmat = np.zeros([matdim,matdim])
    expmat = np.zeros([matdim-1,ntime],dtype='complex')
    cmp = np.zeros([matdim,5],dtype=complex)
    ct = np.ones(len(timeax))*S2

    #construct rate matrix
    # negative values of i are wrapped to end of matrix, as per Python
    for i in range(int(-Mdim/2),int(Mdim/2+1)):
        rmat[i,i] = i**2
        if i > -Mdim/2:
            rmat[i,i-1] = -i*e0/4
        if i < Mdim/2 :
            rmat[i,i+1] = i*e0/4
    rmat = rmat[1:,1:]
    
    eigsyst = np.linalg.eig(rmat)
    lamp = eigsyst[0]
    vmat = eigsyst[1]
    vinv = np.linalg.inv(vmat)

    # calculate all the exponentials
    for i, lamval in enumerate(lamp):
        expmat[i,:] = np.exp(-lamval*timeax)
    
    for m in range(-2,3):
        for p in range(int(-Mdim/2),int(Mdim/2+1)):
            #cmp[p,m] = iv(abs(p-m),e0/2)/iv(0,e0/2) - (iv(abs(m),e0/2)/iv(0,e0/2))*(iv(abs(p),e0/2)/iv(0,e0/2))
            cmp[p,m] = BesselRatio(abs(p-m),e0) - BesselRatio(abs(m),e0)*BesselRatio(abs(p),e0)
            cmp[p,m] = cmp[p,m]*yarray[m]
    cmp = cmp[1:,1:]
    
    for i in range(0,ntime):
        g = vmat@np.diag(expmat[:,i])@vinv@cmp
        for j in range(-2,2):
            ct[i] = ct[i] + np.real(g[j,j])

    return ct

def vonMiseste(thetaD,e0,Mdim=20):
    # returned value is unitless = Dtau_e
    
    # M matrix should have even dimensions
    matdim = Mdim + 1
    if matdim % 2 == 0:
        matdim = matdim+1
        
    thetaR = thetaD*np.pi/180.0
    y20 = (3*np.cos(thetaR)**2-1)/2
    y21 = np.sqrt(3.0/2.0)*np.sin(thetaR)*np.cos(thetaR)
    y22 = np.sqrt(3.0/8.0)*np.sin(thetaR)**2
    yarray = np.array([y20**2,y21**2,y22**2,y22**2,y21**2])
    
    S2 = vonMisesS2(thetaD,e0)

    rmat = np.zeros([matdim,matdim])
    cmp = np.zeros([matdim,5],dtype=complex)
    
    #construct rate matrix
    # negative values of i are wrapped to end of matrix, as per Python
    for i in range(int(-Mdim/2),int(Mdim/2+1)):
        rmat[i,i] = i**2
        if i > -Mdim/2:
            rmat[i,i-1] = -i*e0/4
        if i < Mdim/2 :
            rmat[i,i+1] = i*e0/4
    rmat = rmat[1:,1:]

    for m in range(-2,3):
        for p in range(int(-Mdim/2),int(Mdim/2+1)):
            #cmp[p,m] = iv(abs(p-m),e0/2)/iv(0,e0/2) - (iv(abs(m),e0/2)/iv(0,e0/2))*(iv(abs(p),e0/2)/iv(0,e0/2))
            cmp[p,m] = BesselRatio(abs(p-m),e0) - BesselRatio(abs(m),e0)*BesselRatio(abs(p),e0)
            cmp[p,m] = cmp[p,m]*yarray[m]
    cmp = cmp[1:,1:]
    
    Minv = np.linalg.inv(rmat)
    g = np.real(Minv@cmp)
    
    taue = 0
    for j in range(-2,2):
        taue = taue + g[j,j]
    taue = taue/(1-S2)

    return taue


In [ ]:
fig = plt.figure(figsize=(4,8))

thetaD = 109
e0 =2.5
timeaxis= np.linspace(0,10,1000) 
tarray = np.linspace(-np.pi,np.pi,500)
vonM = 0.5*e0*(1-np.cos(tarray))

plt.subplot(211,aspect=360/2.5)

plt.plot(tarray*180/np.pi,vonM,c='black')
plt.xlim(-180,180)
plt.ylim(0,2.5)
plt.xlabel(r'$\phi$',fontsize=14)
plt.ylabel(r'$V(\phi)/(k_BT)$',fontsize=14)
plt.text(0.9*360-180,0.05*2.6,"(a)",fontsize=12)

plt.subplot(212,aspect=3)

Ct = vonMisesACF(timeaxis,thetaD,e0,Mdim=20)
taue = vonMiseste(thetaD,e0,Mdim=20)
S2 = vonMisesS2(thetaD,e0)

print("S2:",S2)
print("tau_e:",taue)

plt.axhline(S2,c='black',ls=":")
plt.plot(timeaxis,S2 + (1-S2)*np.exp(-timeaxis/taue),c=ReddishPurple,ls='--')
plt.plot(timeaxis,Ct,c='black')

plt.xlabel(r'$D\tau$',fontsize=14)
plt.ylabel(r'$C(\tau)$',fontsize=14)
plt.xlim(0,3)
plt.ylim(0,1)
plt.text(0.9*3,0.9,"(b)",fontsize=12)


plt.tight_layout()

plt.savefig("Figure1_DiffonCone_ACF.png",dpi=300,pad_inches=0)
plt.savefig("Figure1_DiffonCone_ACF.eps",pad_inches=0)

plt.show()

In [ ]:
sd0 = np.linspace(1,250,500)
e0 = 2*(sd0*np.pi/180)**-2
print(min(e0),max(e0))
t0 = 109

S2array = np.zeros(len(e0))
tearray = np.zeros(len(e0))
S2GAFarray = np.zeros(len(e0))

S2 = vonMisesS2(t0,0)
print(S2)

for i,e00 in enumerate(e0):
    S2array[i] = vonMisesS2(t0,e00)
    tearray[i] = vonMiseste(t0,e00,Mdim=40)
    S2GAFarray[i] = GAF(t0,(sd0[i]*np.pi/180)**2)
        
fig = plt.figure(figsize=(4,12))

plt.subplot(311,aspect=180)

plt.axhline(S2,c='black',ls=":")
plt.plot(sd0,S2array,c='black')
plt.plot(sd0,S2GAFarray,c=ReddishPurple,ls='--')
plt.xlim(0,180)
plt.ylim(0,1)
plt.xlabel(r'$\sigma$',fontsize=14)
plt.ylabel(r'$S^2$',fontsize=14)
plt.text(0.9*180,0.9,"(a)",fontsize=12)

plt.subplot(312,aspect=180/0.5)

plt.plot(sd0,tearray,c='black')
plt.xlim(0,180)
plt.ylim(0,0.5)
plt.xlabel(r'$\sigma$',fontsize=14)
plt.ylabel(r'$D\tau_e$',fontsize=14)
plt.text(0.9*180,0.45,"(b)",fontsize=12)

plt.subplot(313, aspect=2)

xfit = S2array-S2
pfit = np.polynomial.polynomial.Polynomial.fit(xfit,tearray,deg=2)
print(pfit)
xyfit = pfit.linspace(n=100)
plt.plot(S2array,tearray,c='black')
#plt.plot(xyfit[0]+S2,xyfit[1],c=ReddishPurple,ls="--")
plt.axvline(S2,c='black',ls=":")
plt.xlim(0,1)
plt.ylim(0,0.5)
plt.xlabel(r'$S^2$',fontsize=14)
plt.ylabel(r'$D\tau_e$',fontsize=14)
plt.text(0.9,0.45,"(c)",fontsize=12)

plt.tight_layout()

plt.savefig("Figure2_DiffonCone_V1.png",dpi=300,pad_inches=0)
plt.savefig("Figure2_DiffonCone_V1.eps",pad_inches=0)
    

plt.show()

In [ ]:
sd0 = np.linspace(3,250,500)
e0 = 2*(sd0*np.pi/180)**-2
print(min(e0),max(e0))
t0 = 109

S2array = np.zeros(len(e0))
Entropyarray = np.zeros(len(e0))


for i,e00 in enumerate(e0):
    S2array[i] = vonMisesS2(t0,e00)
    Entropyarray[i] = vonMisesEntropy(e00)

def ringSp(Sls2,a,b,c,d):
    x = 1-Sls2
    y = a+(b*x)/(1+c*x+d*x**2)
    return y


ringSppars,pcov = curve_fit(ringSp,S2array[Entropyarray>-1],Entropyarray[Entropyarray>-1],p0=[-2,2,1,-1])
print("Entropy fit",ringSppars)

ringSpfit = ringSp(S2array,*ringSppars)

t0 = 109
S2 = vonMisesS2(t0,0)
for i,e00 in enumerate(e0):
    S2array[i] = vonMisesS2(t0,e00)
    Entropyarray[i] = vonMisesEntropy(e00)

ringSpfit = ringSp(S2array,*ringSppars)

fig = plt.figure(figsize=(4,4))

#plt.subplot(111,aspect=180)

plt.plot((S2array-S2)/(1-S2),Entropyarray,c='black')
plt.plot((S2array-S2)/(1-S2),ringSpfit,c=ReddishPurple)
plt.ylim(-1,2)
plt.xlim(0,1)
plt.ylabel(r'$S/k_B$',fontsize=14)
plt.xlabel(r'$S^2-Y_2^0(\theta_0)$',fontsize=14)

plt.tight_layout()

plt.show()

## 3-well potential

In [ ]:
# Numerical calculations of averages

def V3potential(tarray,thetaD,e0,cnarray):
    thetaR = thetaD*np.pi/180
    V = e0*(cnarray[0] + 2*cnarray[1]*np.cos(tarray) + 2*cnarray[2]*np.cos(2*tarray) + 2*cnarray[3]*np.cos(3*tarray))
    return V

def V3pdist(tarray,thetaD,e0,cnarray):
    thetaR = thetaD*np.pi/180
    V = V3potential(tarray,thetaD,e0,cnarray)
    P = np.exp(-V)
    Z = (np.sum(2*P)-P[0]-P[-1])*2*np.pi/(2*len(tarray))
    P = P/Z
    return P

def V3entropy(tarray,thetaD,e0,cnarray):
    thetaR = thetaD*np.pi/180
    V = V3potential(tarray,thetaD,e0,cnarray)
    P = np.exp(-V)
    Z = (np.sum(2*P)-P[0]-P[-1])*2*np.pi/(2*len(tarray))
    P = P/Z
    plnp = P*V
    Sentropy = (np.sum(2*plnp)-plnp[0]-plnp[-1])*2*np.pi/(2*len(tarray))
    Sentropy = Sentropy + np.log(Z)
    return Sentropy
    
def V3S2num(tarray,thetaD,e0,cnarray):
    thetaR = thetaD*np.pi/180
    Y0 = (3*np.cos(thetaR)**2-1)/2
    Y1 = -np.sqrt(3.0/2.0)*np.sin(thetaR)*np.cos(thetaR)
    Y2 = np.sqrt(3.0/8.0)*np.sin(thetaR)**2
    Ymat = np.array([Y0**2,Y1**2,Y2**2,Y2**2,Y1**2])
    S2 = Y0**2

    P = V3pdist(tarray,thetaD,e0,cnarray)
    
    for i in range(-2,3):
        if i != 0:
            tmp = np.exp(i*1j*tarray)*P
            Mtmp = (np.sum(2*tmp)-tmp[0]-tmp[-1])*2*np.pi/(2*len(tarray))
            S2 = S2 + Mtmp*np.conjugate(Mtmp)*Ymat[i]
    
    return np.real(S2)

def V3S2jump(tarray,thetaD,e0,cnarray):
    thetaR = thetaD*np.pi/180.0
    Y0 = (3*np.cos(thetaR)**2 - 1)/2
    Y1 = -np.sqrt(3.0/2.0)*np.sin(thetaR)*np.cos(thetaR)
    Y2 = np.sqrt(3.0/8.0)*np.sin(thetaR)**2

    S2 = 0

    P = V3pdist(tarray,thetaD,e0,cnarray)
    
    thetaTrans = 0
    thetaPlus = 0
    thetaMinus = 0
    cos2theta = (3*cnarray[3]-cnarray[1])/(12*cnarray[3])
    if cos2theta > 0:
        thetaRotamer = np.arccos(np.sqrt(cos2theta))
        thetaPlus = thetaRotamer-np.pi
        thetaMinus = np.pi - thetaRotamer
        thetaMax1 = -thetaRotamer
        thetaMax2 = thetaRotamer
        
    for i,val in enumerate(tarray):
        if val - np.pi < thetaMax1 :
            cut1 = i
        elif val - np.pi < thetaMax2:
            cut2 = i
    Pplus = (np.sum(2*P[0:int(cut1)])-P[0]-P[int(cut1)])*2*np.pi/(2*len(tarray))
    Ptrans = (np.sum(2*P[int(cut1):int(cut2)])-P[int(cut1)]-P[cut2])*2*np.pi/(2*len(tarray))
    Pminus = (np.sum(2*P[int(cut2):])-P[cut2]-P[-1])*2*np.pi/(2*len(tarray))
    
    M0 = 1
    M1 = Pminus*np.exp(1j*thetaMinus)+Ptrans*np.exp(1j*thetaTrans)+Pplus*np.exp(1j*thetaPlus)
    Mm1 = Pminus*np.exp(-1j*thetaMinus)+Ptrans*np.exp(-1j*thetaTrans)+Pplus*np.exp(-1j*thetaPlus)
    M2 = Pminus*np.exp(2*1j*thetaMinus)+Ptrans*np.exp(2*1j*thetaTrans)+Pplus*np.exp(2*1j*thetaPlus)
    Mm2 = Pminus*np.exp(-2*1j*thetaMinus)+Ptrans*np.exp(-2*1j*thetaTrans)+Pplus*np.exp(-2*1j*thetaPlus)
    
    S2 = Y0**2 + 2*Y1**2*M1*Mm1 + 2*Y2**2*M2*Mm2
    S2 = np.real(S2)
    
    return S2

def V3ACF(timeax,tarray,thetaD,e0,cnarray,Mdim=20):
    # timeax is unitless = Dt
    
    thetaR = thetaD*np.pi/180.0
    Y0 = (3*np.cos(thetaR)**2-1)/2
    Y1 = -np.sqrt(3.0/2.0)*np.sin(thetaR)*np.cos(thetaR)
    Y2 = np.sqrt(3.0/8.0)*np.sin(thetaR)**2
    yarray = np.array([Y0**2,Y1**2,Y2**2,Y2**2,Y1**2])
    
    S2 = V3S2num(tarray,thetaD,e0,cnarray)
    
    # calculate equilibrium averages
    P = V3pdist(tarray,thetaD,e0,cnarray)

    # M matrix should have even dimensions
    matdim = Mdim + 1
    if matdim % 2 == 0:
        matdim = matdim+1
        
    ntime = len(timeax)
    rmat = np.zeros([matdim,matdim])
    expmat = np.zeros([matdim-1,ntime],dtype='complex')
    cmp = np.zeros([matdim,5],dtype=complex)
    ct = np.ones(len(timeax))*S2

    #construct rate matrix
    # negative values of i are wrapped to end of matrix, as per Python
    for i in range(int(-Mdim/2),int(Mdim/2+1)):
        rmat[i,i] = i**2
        if i > -Mdim/2:
            rmat[i,i-1] = i*e0*cnarray[1]
            if i > -Mdim/2 + 1:
                rmat[i,i-2] = i*2*e0*cnarray[2]
            if i > -Mdim/2 + 2:
                rmat[i,i-3] = i*3*e0*cnarray[3]
        if i < Mdim/2 :
            rmat[i,i+1] = -i*e0*cnarray[1]
            if i < Mdim/2 - 1:
                rmat[i,i+2] = -i*2*e0*cnarray[2]
            if i < Mdim/2 - 2:
                rmat[i,i+3] = -i*3*e0*cnarray[3]
    rmat = rmat[1:,1:]
    eigsyst = np.linalg.eig(rmat)
    lamp = eigsyst[0]
    vmat = eigsyst[1]
    vinv = np.linalg.inv(vmat)

    # calculate all the exponentials
    for i, lamval in enumerate(lamp):
        expmat[i,:] = np.exp(-lamval*timeax)
    
    for m in range(-2,3):
        tmp = np.exp(-m*1j*tarray)*P
        Mm = (np.sum(2*tmp)-tmp[0]-tmp[-1])*2*np.pi/(2*len(tarray)) 
        for p in range(int(-Mdim/2),int(Mdim/2+1)):
            tmp = np.exp(p*1j*tarray)*P
            Mp = (np.sum(2*tmp)-tmp[0]-tmp[-1])*2*np.pi/(2*len(tarray))
            
            tmp = np.exp((p-m)*1j*tarray)*P
            Mmp = (np.sum(2*tmp)-tmp[0]-tmp[-1])*2*np.pi/(2*len(tarray))
            
            cmp[p,m] = (Mmp - Mm*Mp)*yarray[m]
    cmp = cmp[1:,1:]
    
    for i in range(0,ntime):
        g = vmat@np.diag(expmat[:,i])@vinv@cmp
        for j in range(-2,2):
            ct[i] = ct[i] + np.real(g[j,j])
    return ct
   
def V3te(tarray,thetaD,e0,cnarray,Mdim=20):
    # the effective correlation time is dimensionless unitless = Dtau_e
    
    thetaR = thetaD*np.pi/180.0
    Y0 = (3*np.cos(thetaR)**2-1)/2
    Y1 = np.sqrt(3.0/2.0)*np.sin(thetaR)*np.cos(thetaR)
    Y2 = np.sqrt(3.0/8.0)*np.sin(thetaR)**2
    yarray = np.array([Y0**2,Y1**2,Y2**2,Y2**2,Y1**2])
    
    S2 = V3S2num(tarray,thetaD,e0,cnarray)
    
    # calculate equilibrium averages
    P = V3pdist(tarray,thetaD,e0,cnarray)
    
    # M matrix should have odd dimensions
    matdim = Mdim + 1
    if matdim % 2 == 0:
        matdim = matdim+1
   
    rmat = np.zeros([matdim,matdim])
    cmp = np.zeros([matdim,5],dtype=complex)
    
    #construct rate matrix
    # negative values of i are wrapped to end of matrix, as per Python
    for i in range(int(-Mdim/2),int(Mdim/2+1)):
        rmat[i,i] = i**2
        if i > -Mdim/2:
            rmat[i,i-1] = i*e0*cnarray[1]
            if i > -Mdim/2 + 1:
                rmat[i,i-2] = i*2*e0*cnarray[2]
            if i > -Mdim/2 + 2:
                rmat[i,i-3] = i*3*e0*cnarray[3]
        if i < Mdim/2 :
            rmat[i,i+1] = -i*e0*cnarray[1]
            if i < Mdim/2 - 1:
                rmat[i,i+2] = -i*2*e0*cnarray[2]
            if i < Mdim/2 - 2:
                rmat[i,i+3] = -i*3*e0*cnarray[3]
    rmat = rmat[1:,1:]

    for m in range(-2,3):
        tmp = np.exp(-m*1j*tarray)*P
        Mm = (np.sum(2*tmp)-tmp[0]-tmp[-1])*2*np.pi/(2*len(tarray)) 
        
        for p in range(int(-Mdim/2),int(Mdim/2+1)):
            tmp = np.exp(p*1j*tarray)*P
            Mp = (np.sum(2*tmp)-tmp[0]-tmp[-1])*2*np.pi/(2*len(tarray))
            
            tmp = np.exp((p-m)*1j*tarray)*P
            Mmp = (np.sum(2*tmp)-tmp[0]-tmp[-1])*2*np.pi/(2*len(tarray))
            
            cmp[p,m] = (Mmp - Mm*Mp)*yarray[m]
    cmp = cmp[1:,1:]
    
    Minv = np.linalg.inv(rmat)
    g = np.real(Minv@cmp)
    
    taue = 0
    for j in range(-2,2):
        taue = taue + g[j,j]
    taue = taue/(1-S2)
    
    return taue
   

In [ ]:
fig = plt.figure(figsize=(4,8))

thetaD = 109
e0 = 2.5
e0vm = 1.776
cnarray = np.array([1/2.0,3/20.0,0,2/20.0])
#cnarray = np.array([1/2.0,4/20.0,0,1/20.0])

cn1array = np.array([1/2.0,-1/4.0,0,0])

timeaxis= np.linspace(0,10,5000) 
tarray = np.linspace(0,2*np.pi,5000)

V3p = V3potential(tarray,thetaD,e0,cnarray)
S2 = V3S2num(tarray,thetaD,e0,cnarray)

S2vm = vonMisesS2(thetaD,e0vm)

plt.subplot(211,aspect=360/e0)
plt.plot(tarray*180/np.pi,V3p,c='black')
plt.xlim(0,360)
plt.ylim(0,e0)
plt.xlabel(r'$\phi$',fontsize=14)
plt.ylabel(r'$V(\phi)/(k_BT)$',fontsize=14)
plt.text(0.9*360,0.075,"(a)",fontsize=12)

plt.subplot(212,aspect=3)

# Series expansion calculation
CtV3 = V3ACF(timeaxis,tarray,thetaD,e0,cnarray,Mdim=20)
taueV3 = V3te(tarray,thetaD,e0,cnarray,Mdim=20)
CtV1 = V3ACF(timeaxis,tarray-np.pi,thetaD,e0vm,cn1array,Mdim=20)
taueV1 = V3te(tarray-np.pi,thetaD,e0vm,cn1array,Mdim=20)

pfit,pcov = curve_fit(extLS,timeaxis,CtV3,p0=[S2,0.8,1,10])
print("extLS fit",pfit)
yfit = extLS(timeaxis,*pfit)

print(CtV3[0],CtV3[-1])

CtvonMises = vonMisesACF(timeaxis,thetaD,e0vm,Mdim=30)
tauevonMises = vonMiseste(thetaD,e0vm,Mdim=30)

print("S2:",S2,S2vm)
print("tau_e:",taueV3,tauevonMises,taueV1)

plt.axhline(S2,c='black',ls=":")
plt.plot(timeaxis,CtV1,c=ReddishPurple,ls='--')
plt.plot(timeaxis,S2 + (1-S2)*np.exp(-timeaxis/taueV3),c=Blue,ls='-.')
plt.plot(timeaxis,CtV3,c='black')
#plt.plot(timeaxis,yfit,c=Green,ls=':')


plt.xlabel(r'$D\tau$',fontsize=14)
plt.ylabel(r'$C(\tau)$',fontsize=14)
plt.xlim(0,3)
plt.ylim(0,1)
plt.text(0.9*3,0.9,"(b)",fontsize=12)

plt.tight_layout()

plt.savefig("Figure3_DiffonConeV3_ACF.png",dpi=300,pad_inches=0)
plt.savefig("Figure3_DiffonConeV3_ACF.eps",pad_inches=0)

plt.show()

In [ ]:
fig = plt.figure(figsize=(12,12))

cnarray = np.array([[1/2.0,4/20.0,0,1/20.0],
                     [1/2.0,3/20.0,0,2/20.0],
                     [1/2.0,1/20.0,0,4/20.0]])

cn1array = np.array([1/2.0,-1/4.0,0,0])

tarray = np.linspace(0,2*np.pi,5000)

sd0 = np.linspace(3.0,250,500)
e0 = 2*(sd0*np.pi/180)**-2

t0 = 109
ep0 = 10

S2array = np.zeros(len(e0))
V3tearray = np.zeros(len(e0))
S2GAFarray = np.zeros(len(e0))
S2V3array = np.zeros(len(e0))
S2V3jumparray = np.zeros(len(e0))

V1pot = V3potential(tarray-np.pi,thetaD,ep0,cn1array)
V1prob = V3pdist(tarray-np.pi,thetaD,ep0,cn1array)
V1S2 = V3S2num(tarray-np.pi,thetaD,ep0,cn1array)
V1taue = V3te(tarray-np.pi,thetaD,ep0,cn1array)

glab = ["(a)","(b)","(c)","(d)","(e)","(f)","(g)","(h)","(i)"]

for indx in range(0,3):
    
    plt.subplot(3,3,indx*3+1,aspect=360/ep0)

    V3pot = V3potential(tarray,thetaD,ep0,cnarray[indx,:])
    V3prob = V3pdist(tarray,thetaD,ep0,cnarray[indx,:])
    V3S2 = V3S2num(tarray,thetaD,ep0,cnarray[indx,:])
    
    cos2theta = (3*cnarray[indx,3]-cnarray[indx,1])/(12*cnarray[indx,3])
    if cos2theta > 0:
        thetaPlus = np.arccos(np.sqrt(cos2theta))*180/np.pi
    
    plt.plot(tarray*180/np.pi,V3pot,c='black')
    plt.plot(tarray*180/np.pi,V1pot,c=ReddishPurple,ls='--')
    plt.axvline(180,c=Blue,ls=":")

    if cos2theta > 0:
        plt.axvline(thetaPlus,c=Blue,ls=":")
        plt.axvline(360-thetaPlus,c=Blue,ls=":")
    plt.xlim(0,360)
    plt.ylim(0,ep0)
    if indx ==2:
        plt.xlabel(r'$\phi$',fontsize=14)
    plt.ylabel(r'$V(\phi)/(k_BT)$',fontsize=14)
    plt.text(0.05*360,0.25,glab[indx*3],fontsize=12)

    plt.subplot(3,3,indx*3+2,aspect=360/1.7)
    plt.plot(tarray*180/np.pi,V3prob,c='black')
    plt.plot(tarray*180/np.pi,V1prob,c=ReddishPurple,ls='--')
    plt.axvline(180,c=Blue,ls=":")
    if cos2theta > 0:
        plt.axvline(thetaPlus,c=Blue,ls=":")
        plt.axvline(360-thetaPlus,c=Blue,ls=":")
    plt.xlim(0,360)
    plt.ylim(0,1.7)
    if indx ==2:
        plt.xlabel(r'$\phi$',fontsize=14)
    plt.ylabel(r'$p(\phi)$',fontsize=14)
    plt.text(0.9*360,0.9*1.7,glab[indx*3+1],fontsize=12)

    plt.subplot(3,3,indx*3+3,aspect=180/1.05)
    for i,e00 in enumerate(e0):
        S2array[i] = vonMisesS2(t0,e00)
        S2V3array[i] = V3S2num(tarray,t0,e00,cnarray[indx,:])
        V3tearray[i] =  V3te(tarray,t0,e00,cnarray[indx,:],Mdim=20)
        if cos2theta > 0:
            S2V3jumparray[i] = V3S2jump(tarray,t0,e00,cnarray[indx,:])
        S2GAFarray[i] = GAF(t0,(sd0[i]*np.pi/180)**2)
        
    plt.plot(2/np.sqrt(e0)*180/np.pi,S2array,c=ReddishPurple,ls='--')
    #plt.plot(1/e0,S2GAFarray,c=Green)
    plt.plot(2/np.sqrt(e0)*180/np.pi,S2V3array,c='black')
    if cos2theta > 0:
        plt.plot(2/np.sqrt(e0)*180/np.pi,S2V3jumparray,c=Blue,ls='-.')
    plt.xlim(0,180)
    plt.ylim(0,1.05)
    if indx ==2:
        plt.xlabel(r'$\sigma$',fontsize=14)
    plt.ylabel(r'$S^2$',fontsize=14)
    plt.text(0.9*180,0.9*1.05,glab[indx*3+2],fontsize=12)

plt.tight_layout()

plt.savefig("Figure4_V3_V1_compare.png",dpi=300,pad_inches=0)
plt.savefig("Figure4_V3_V1_compare.eps",pad_inches=0)

plt.show()

In [ ]:
fig = plt.figure(figsize=(4,8))

thetaD = 109
e0 = 10
e0vm = 1.776
cnarray = np.array([1/2.0,3/20.0,0,2/20.0])
#cnarray = np.array([1/2.0,3/16.0,0,1/16.0])

cn1array = np.array([1/2.0,-1/4.0,0,0])

timeaxis= np.linspace(0,10,5000) 
tarray = np.linspace(0,2*np.pi,5000)

V3p = V3potential(tarray,thetaD,e0,cnarray)
S2 = V3S2num(tarray,thetaD,e0,cnarray)

S2vm = vonMisesS2(thetaD,e0vm)

plt.subplot(211,aspect=360/e0)
plt.plot(tarray*180/np.pi,V3p,c='black')
plt.xlim(0,360)
plt.ylim(0,e0)
plt.xlabel(r'$\phi$',fontsize=14)
plt.ylabel(r'$V(\phi)/(k_BT)$',fontsize=14)
plt.text(0.9*360,0.075*10/2.5,"(a)",fontsize=12)

plt.subplot(212,aspect=3/0.25)

# Series expansion calculation
CtV3 = V3ACF(timeaxis,tarray,thetaD,e0,cnarray,Mdim=30)
taueV3 = V3te(tarray,thetaD,e0,cnarray,Mdim=30)
CtV1 = V3ACF(timeaxis,tarray-np.pi,thetaD,e0vm,cn1array,Mdim=30)
taueV1 = V3te(tarray-np.pi,thetaD,e0vm,cn1array,Mdim=30)

CtvonMises = vonMisesACF(timeaxis,thetaD,e0vm,Mdim=30)
tauevonMises = vonMiseste(thetaD,e0vm,Mdim=30)

print("S2:",S2,S2vm)
print("tau_e:",taueV3,tauevonMises,taueV1)

pfit,pcov = curve_fit(extLS,timeaxis,CtV3,p0=[S2,0.8,1,10])
print("extLS fit",pfit)
yfit = extLS(timeaxis,*pfit)

plt.axhline(S2,c='black',ls=":")
plt.plot(timeaxis,CtV3,c='black')
plt.plot(timeaxis,S2 + (1-S2)*np.exp(-timeaxis/taueV3),c=Blue,ls='-.')
plt.plot(timeaxis,yfit,c=ReddishPurple,ls="--")
plt.xlabel(r'$D\tau$',fontsize=14)
plt.ylabel(r'$C(\tau)$',fontsize=14)
plt.xlim(0,3)
plt.ylim(0.75,1)
plt.text(0.9*3,0.9*0.25+0.75,"(b)",fontsize=12)


plt.tight_layout()

plt.savefig("Figure5_DiffonConeV3_ACF_2.png",dpi=300,pad_inches=0)
plt.savefig("Figure5_DiffonConeV3_ACF_2.eps",pad_inches=0)

plt.show()

In [ ]:
fig = plt.figure(figsize=(9,3))

cnarray = np.array([[1/2.0,4/20.0,0,1/20.0],
                     [1/2.0,3/20.0,0,2/20.0],
                     [1/2.0,1/20.0,0,4/20.0]])

tarray = np.linspace(0,2*np.pi,5000)

sd0 = np.linspace(3.0,250,500)
e0 = 2*(sd0*np.pi/180)**-2

t0 = 109
ep0 = 10


EntropyV1array = np.zeros(len(e0))
S2V1array = np.zeros(len(e0))
EntropyV3array = np.zeros(len(e0))
S2V3array = np.zeros(len(e0))

S2 = vonMisesS2(t0,0)

glab = ["(a)","(b)","(c)","(d)","(e)","(f)","(g)","(h)","(i)"]

for indx in range(0,3):
    
    plt.subplot(1,3,indx+1,aspect=0.33333)
    
    for i,e00 in enumerate(e0):
        S2V1array[i] = vonMisesS2(t0,e00)
        EntropyV1array[i] = vonMisesEntropy(e00)
        S2V3array[i] = V3S2num(tarray,t0,e00,cnarray[indx,:])
        EntropyV3array[i] =  V3entropy(tarray,t0,e00,cnarray[indx,:])
        
        #print(S2V3array[i],EntropyV3array[i])

    y = ringSp(S2V3array,*ringSppars)
    
    plt.plot(S2V3array,EntropyV3array,c='black')
    plt.plot(S2V3array,y,c=ReddishPurple)
    plt.xlim(0,1)
    plt.ylim(-1,2)
    if indx ==0:
        plt.ylabel(r'$S_p/k_B$',fontsize=14)
    if indx == 1:
        plt.xlabel(r'$S^2$',fontsize=14)
    plt.text(0.8,1.7,glab[indx],fontsize=12)
    plt.axvline(S2,ls='--',c=Blue)

plt.tight_layout()

plt.savefig("V3_V1_entropypng",dpi=300,pad_inches=0)
plt.savefig("V3_V1_entropy.pdf",pad_inches=0)

plt.show()

## Diffusion in a axially symmetric potential

## Wigner 3-j symbols

In [ ]:
# Calculate Wigner 3-j symbols

def wignerSpecial(j1,j2,m1,m2):
    # assumes j1+j2=j3, m3 = -(m1+m2)
    
    tmpn = np.sqrt(np.math.factorial(2*j1)*np.math.factorial(2*j2)*np.math.factorial(j1+j2+m1+m2)*
                  np.math.factorial(j1+j2-m1-m2))
    tmpd = np.sqrt((2*j1+2*j2+1)*np.math.factorial(2*j1+2*j2)*np.math.factorial(j1+m1)*
                   np.math.factorial(j1-m1)*np.math.factorial(j2+m2)*np.math.factorial(j2-m2))
    j3 = (-1)**(-j1+j2-m1-m2)*tmpn/tmpd
    return j3

def wigner0j(j,m):
    tmp = (-1)**(j-m)/np.sqrt(2*j+1)
    return tmp

def wigner1j(j,m):
    tmp = np.sqrt((j+1)**2 - m**2)
    tmpd = np.sqrt((2*j+3)*(j+1)*(2*j+1))
    j3 = (-1)**(j-1-m)*tmp/tmpd
    return j3

def wigner3j(k,l,m,verbose=False):
    w3j = 0
    if k == 0 and l == 2 and m == 0:
        c0 = np.sqrt(1/5.0)
        cm = c0
        w3j = c0*cm
    elif k == 1 and l == 1:
        if m == 0:
            c0 = np.sqrt(2.0/15)
            cm = c0
            w3j = c0*cm
        elif m == 1 or m == -1:
            c0 = np.sqrt(2/15.0)
            cm = -np.sqrt(1/10)
            w3j = c0*cm
    elif k == 1 and l == 3:
        if m == 0:
            c0 = -np.sqrt(3.0/35)
            cm = c0
            w3j = c0*cm
        elif m == 1 or m == -1:
            c0 = -np.sqrt(3.0/35)
            cm = -np.sqrt(1/35.0)
            w3j = c0*cm
    elif k >= 2:
        if l == k-2:
            tmp = (2*k+1)*(2*k)*(2*k-1)*(2*k-2)*(2*k-3)
            c0 = (-1)**(k-2)*np.sqrt(6/tmp)*k*(k-1)
            if m == 0:
                cm = c0
                w3j = c0*cm
            elif m == 1 or m == -1:
                tmp = (2*k+1)*(2*k)*(2*k-1)*(2*k-2)*(2*k-3)
                cm = (-1)**(k-1)*np.sqrt(4*(k+1)*k*(k-1)**2/tmp)
                w3j = c0*cm
            elif m == 2 or m == -2:
                tmp = (2*k+1)*(2*k)*(2*k-1)*(2*k-2)*(2*k-3)
                cm = (-1)**(k-2)*np.sqrt((k+2)*(k+1)*k*(k-1)/tmp)
                w3j = c0*cm
        elif l == k:
            tmp = (2*k+3)*(2*k+2)*(2*k+1)*(2*k)*(2*k-1)
            c0 = (-1)**(k-1)*np.sqrt(1/tmp)*2*k*(k+1)
            if m == 0:
                cm = c0
                w3j = c0*cm
            elif m == 1 or m == -1:
                tmp = (2*k+3)*(2*k+2)*(2*k+1)*(2*k)*(2*k-1)
                cm = (-1)**(k-2)*np.sqrt(6*(k+1)*k/tmp)
                w3j = c0*cm
            elif m == 2 or m == -2:
                tmp = (2*k+3)*(2*k+2)*(2*k+1)*(2*k)*(2*k-1)
                cm = (-1)**(k-2)*np.sqrt(6*(k+2)*(k+1)*k*(k-1)/tmp)
                w3j = c0*cm
        elif l == k+2:
            tmp = (2*k+5)*(2*k+4)*(2*k+3)*(2*k+2)*(2*k+1)
            c0 = (-1)**(k-2)*np.sqrt(6/tmp)*(k+2)*(k+1)
            if m == 0:
                cm = c0
                w3j = c0*cm
            elif m == 1 or m == -1:
                tmp = (2*k+5)*(2*k+4)*(2*k+3)*(2*k+2)*(2*k+1)
                cm = (-1)**(k-2)*np.sqrt(4*(k+2)**2*(k+1)*k/tmp)
                w3j = c0*cm
            elif m == 2 or m == -2:
                tmp = (2*k+5)*(2*k+4)*(2*k+3)*(2*k+2)*(2*k+1)
                cm = (-1)**(k-2)*np.sqrt((k+2)*(k+1)*k*(k-1)/tmp)
                w3j = c0*cm
    else:
        w3j = 0
        c0 = 0
        cm = 0
    
    if verbose:
        return w3j,c0,cm
    else:
        return w3j


## Axial potential autocorrelation functions

In [ ]:

def avetheta(eps0):
    temp = iv(0,eps0/2)-np.exp(-eps0/2)
    temp = np.pi*temp/(2*np.sinh(eps0/2))
    return temp

def axialPotential(thetaD,e0):
    thetaR = thetaD*np.pi/180
    V = (e0/2)*(1-np.cos(thetaR))
    return V

# recursive calculation of mean values
def Pbar(e0,n):
    parray = np.zeros(n)
    parray[0] = 1
    
    if e0 > 0:
        if n >= 2:
            parray[1] = np.cosh(e0/2)/np.sinh(e0/2) - 2/e0
        if n >= 3:
            for i in range(1,n-1):
                parray[i+1] = -2*(2*i+1)/e0*parray[i] + parray[i-1]
    return parray

def PbarRecur(e0,nmax):
    # Only for Varray = 2/3,-1/2,-1/6
    Pbar = np.zeros(nmax)
    z = np.sqrt(np.pi/e0)*np.exp(-e0)*erfi(np.sqrt(e0))
    
    Pbar[0] = 1
    Pbar[1] = (np.exp(e0)-1)*(2/np.sqrt(np.pi*e0))/erfi(np.sqrt(e0)) -1

    for n in range(1,nmax-1):
        n2 = int(np.floor((n-1)/2))
        
        Psum = 0
        for k in range(0,n2+1):
            if n % 2 == 0:
                Psum = Psum + Pbar[2*k+1]*(4*k+3)
            else:
                Psum = Psum + Pbar[2*k]*(4*k+1)
                
        Ptmp = -(2*n+1)*Pbar[n] - n*Pbar[n-1]
        Ptmp = Ptmp + (2/np.sqrt(np.pi*e0))*(2*n+1)*(np.exp(e0)-(-1)**n)/erfi(np.sqrt(e0))
        Ptmp = Ptmp -(2/e0)*(2*n+1)*Psum
        Ptmp = Ptmp/(n+1)
        Pbar[n+1] = Ptmp
    
    return Pbar

def PbarRecurAll(e0,Vnarray,nmax):
    V = np.absolute(Vnarray)
    Pbar = np.zeros(nmax)
    
    Z = np.sqrt(np.pi/(6*V[2]*e0))*np.exp(-e0*(3*V[2]+V[1])**2/(6*V[2]))
    Z = Z*(erfi((3*V[2]+V[1])*np.sqrt(e0/(6*V[2]))) - erfi((-3*V[2]+V[1])*np.sqrt(e0/(6*V[2]))))

    Pbar[0] = 1
    Pbar[1] = (1/(3*e0*V[2]))*(1-np.exp(-2*V[1]*e0))/Z - V[1]/(3*V[2])

    for n in range(1,nmax-1):
        n2 = int(np.floor((n-1)/2))
        
        Psum = 0
        for k in range(0,n2+1):
            if n % 2 == 0:
                Psum = Psum + Pbar[2*k+1]*(4*k+3)
            else:
                Psum = Psum + Pbar[2*k]*(4*k+1)
                
        Ptmp = -((2*n+1)*V[1]/(3*V[2]))*Pbar[n] - n*Pbar[n-1]
        Ptmp = Ptmp + ((2*n+1)/(3*V[2]*e0))*(1-np.exp(-2*V[1]*e0)*(-1)**n)/Z
        Ptmp = Ptmp -((2*n+1)/(3*e0*V[2]))*Psum
        Ptmp = Ptmp/(n+1)
        Pbar[n+1] = Ptmp
    
    return Pbar


def Pbarnum(e0,Vnarray,lmax):
    
    xarray = np.linspace(-1,1,10000)
    P0 = 1
    P1 = xarray
    P2 = (3*xarray**2-1)/2
    V = Vnarray[0]*P0 + Vnarray[1]*P1 + Vnarray[2]*P2
    
    #calculate probability distribtion
    tmp = np.exp(-e0*V)
    Z = (np.sum(2*tmp)-tmp[0]-tmp[-1])*2/(2*len(xarray))
    Pdist = tmp/Z
    
    Pbar = np.zeros(lmax)
    for i in range(0,lmax):
        tmp = lpmv(0,i,xarray)*Pdist
        Pbar[i] = (np.sum(2*tmp)-tmp[0]-tmp[-1])*2/(2*len(xarray))
    return Pbar

# calculate ACF for cosine potential for Y2m

def axialACF2m(timeax,e0,tc,mmag,Mdim=10):
    
    D = 1/(6*tc)
    
    ACF = np.zeros(len(timeax))
    
    matdim = Mdim + 1
    Pbararray = Pbar(e0,matdim+2)
    
    Y2Ypbar = np.zeros(matdim)
    for p in range(0,matdim):
        if p == 0 and mmag == 0:
            Y2Ypbar[p] = (-1)**mmag*(2*(p+2)+1)*wigner3j(p,p+2,mmag)*Pbararray[p+2]
        elif p == 1 and mmag <= 1:
            Y2Ypbar[p] = (2*p+1)*wigner3j(p,p,mmag)*Pbararray[p] + (2*(p+2)+1)*wigner3j(p,p+2,mmag)*Pbararray[p+2]
            Y2Ypbar[p] = (-1)**mmag*Y2Ypbar[p]
        elif p >= 2:
            Y2Ypbar[p] = (2*(p-2)+1)*wigner3j(p,p-2,mmag)*Pbararray[p-2] + (2*p+1)*wigner3j(p,p,
                        mmag)*Pbararray[p] + (2*(p+2)+1)*wigner3j(p,p+2,mmag)*Pbararray[p+2]
            Y2Ypbar[p] = (-1)**mmag*Y2Ypbar[p]
        if mmag == 0:
            Y2Ypbar[p] = Y2Ypbar[p] - Pbararray[2]*Pbararray[p]
    Y2Ypbar = Y2Ypbar[1:]
    
    ntime = len(timeax)
    rmat = np.zeros([matdim,matdim])
    expmat = np.zeros([matdim-1,ntime],dtype='complex')

    #construct rate matrix
    for l in range(0,matdim):
        rmat[l,l] = l*(l+1)
        if l > 0 and l >= mmag:
            rmat[l,l-1] = -(e0/(2*(2*l+1)))*(l+1)*np.sqrt(l**2-mmag**2)
        if l < matdim - 1 and l >= mmag:
            rmat[l,l+1] = (e0/(2*(2*l+1)))*l*np.sqrt((l+1)**2-mmag**2)
    rmat = rmat[1:,1:]
    
    eigsyst = np.linalg.eig(rmat)
    lamp = eigsyst[0]
    vmat = eigsyst[1]
    vinv = np.linalg.inv(vmat)
    
    # calculate all the exponentials
    for i, lamval in enumerate(lamp):
        expmat[i,:] = np.exp(-D*lamval*timeax)
    
    for p in range(0,matdim-1):
        for q in range(0,matdim-1):
            ACF = ACF + vmat[1,q]*vinv[q,p]*expmat[q,:]*Y2Ypbar[p]
        
    if mmag == 0:
        ACF = ACF + Pbararray[2]**2

    return np.real(ACF)

def axialACF(timeax,e0,tc,matdim=10):
    ACF = axialACF2m(taxis,e0,tc,0,matdim) +  2*axialACF2m(taxis,e0,tc,1,matdim) +  2*axialACF2m(taxis,e0,tc,2,matdim)
    return ACF

def axialS2(e0):
    tmp = Pbar(e0,3)
    return tmp[2]**2

def axialte2m(e0,tc,mmag,Mdim):
    
    D = 1/(6*tc)
    taue = 0
    
    matdim = Mdim + 1
    
    Pbararray = Pbar(e0,matdim+2)
    
    Y2Ypbar = np.zeros(matdim)
    for p in range(0,matdim):
        if p == 0 and mmag == 0:
            Y2Ypbar[p] = (-1)**mmag*(2*(p+2)+1)*wigner3j(p,p+2,mmag)*Pbararray[p+2]
        elif p == 1 and mmag <= 1:
            Y2Ypbar[p] = (2*p+1)*wigner3j(p,p,mmag)*Pbararray[p] + (2*(p+2)+1)*wigner3j(p,p+2,mmag)*Pbararray[p+2]
            Y2Ypbar[p] = (-1)**mmag*Y2Ypbar[p]
        elif p >= 2:
            Y2Ypbar[p] = (2*(p-2)+1)*wigner3j(p,p-2,mmag)*Pbararray[p-2] + (2*p+1)*wigner3j(p,p,
                        mmag)*Pbararray[p] + (2*(p+2)+1)*wigner3j(p,p+2,mmag)*Pbararray[p+2]
            Y2Ypbar[p] = (-1)**mmag*Y2Ypbar[p]
        if mmag == 0:
            Y2Ypbar[p] = Y2Ypbar[p] - Pbararray[2]*Pbararray[p]
    Y2Ypbar = Y2Ypbar[1:]
    
    rmat = np.zeros([matdim,matdim])

    #construct rate matrix
    for l in range(0,matdim):
        rmat[l,l] = l*(l+1)
        if l > 0 and l >= mmag:
            rmat[l,l-1] = -(e0/(2*(2*l+1)))*(l+1)*np.sqrt(l**2-mmag**2)
        if l < matdim - 1 and l >= mmag:
            rmat[l,l+1] = (e0/(2*(2*l+1)))*l*np.sqrt((l+1)**2-mmag**2)
    rmat = rmat[1:,1:]
    
    Minv = np.linalg.inv(rmat)
    taue = Minv[1,:]@Y2Ypbar/D
    
    return np.real(taue)

def axialte(e0,tc,Mdim=10):
    S2 = axialS2(e0)
    taue = axialte2m(e0,tc,0,Mdim) +  2*axialte2m(e0,tc,1,Mdim) +  2*axialte2m(e0,tc,2,Mdim)
    taue = taue/(1-S2)
    return taue

# Two-term expansion

def axialPotentialV2(thetaD,e0,Vnarray):
    thetaR = thetaD*np.pi/180
    Y00 = 1.0
    Y10 = np.cos(thetaR)
    Y20 = (3*np.cos(thetaR)**2-1)/2
    
    V = e0*(Y00*Vnarray[0] + Y10*Vnarray[1] + Y20*Vnarray[2])
    return V

def axialACF2mV2(timeax,e0,Vnarray,tc,mmag,Mdim=10):
    
    dVarray = np.array([Vnarray[1],3*Vnarray[2]])
    
    D = 1/(6*tc)
    
    ACF = np.zeros(len(timeax))
    
    matdim = Mdim + 1
    
    Pbararray = PbarRecurAll(e0,Vnarray,matdim+2)
    
    Y2Ypbar = np.zeros(matdim)
    for p in range(0,matdim):
        if p == 0 and mmag == 0:
            Y2Ypbar[p] = (-1)**mmag*(2*(p+2)+1)*wigner3j(p,p+2,mmag)*Pbararray[p+2]
        elif p == 1 and mmag <= 1:
            Y2Ypbar[p] = (2*p+1)*wigner3j(p,p,mmag)*Pbararray[p] + (2*(p+2)+1)*wigner3j(p,p+2,mmag)*Pbararray[p+2]
            Y2Ypbar[p] = (-1)**mmag*Y2Ypbar[p]
        elif p >= 2:
            Y2Ypbar[p] = (2*(p-2)+1)*wigner3j(p,p-2,mmag)*Pbararray[p-2] + (2*p+1)*wigner3j(p,p,
                        mmag)*Pbararray[p] + (2*(p+2)+1)*wigner3j(p,p+2,mmag)*Pbararray[p+2]
            Y2Ypbar[p] = (-1)**mmag*Y2Ypbar[p]
        if mmag == 0:
            Y2Ypbar[p] = Y2Ypbar[p] - Pbararray[2]*Pbararray[p]
    Y2Ypbar = Y2Ypbar[1:]
    
    ntime = len(timeax)
    rmat = np.zeros([matdim,matdim])
    expmat = np.zeros([matdim-1,ntime],dtype='complex')

    #construct rate matrix
    for l in range(0,matdim):
        cll = -3*e0*dVarray[1]*(l*(l+1)-3*mmag**2)/((2*l+3)*(2*l+1))
        rmat[l,l] = l*(l+1) - cll
        
        if l > 0 and l >= mmag:
            rmat[l,l-1] = (e0*dVarray[0]/(2*l+1))*(l+1)*np.sqrt(l**2-mmag**2)
        if l < matdim - 1 and l+1 >= mmag:
            rmat[l,l+1] = -(e0*dVarray[0]/(2*l+1))*l*np.sqrt((l+1)**2-mmag**2)
        if l > 1 and l-1 >= mmag:
            rmat[l,l-2] = (3*e0*dVarray[1]/((2*l+1)*(2*l-1)))*(l+1)*np.sqrt(l**2-mmag**2)*np.sqrt((l-1)**2-mmag**2)
        if l < matdim - 2 and l+1 >= mmag:
            rmat[l,l+2] = -(3*e0*dVarray[1]/((2*l+1)*(2*l+3)))*l*np.sqrt((l+1)**2-mmag**2)*np.sqrt((l+2)**2-mmag**2)
    rmat = rmat[1:,1:]
    
    eigsyst = np.linalg.eig(rmat)
    lamp = eigsyst[0]
    vmat = eigsyst[1]
    vinv = np.linalg.inv(vmat)
    
    # calculate all the exponentials
    for i, lamval in enumerate(lamp):
        expmat[i,:] = np.exp(-D*lamval*timeax)
    
    for p in range(0,matdim-1):
        for q in range(0,matdim-1):
            ACF = ACF + vmat[2,q]*vinv[q,p]*expmat[q,:]*Y2Ypbar[p]
        
    if mmag == 0:
        ACF = ACF + Pbararray[2]**2

    return np.real(ACF)

def axialACFV2(timeax,e0,Vnarray,tc,matdim=10):
    ACF = axialACF2mV2(taxis,e0,Vnarray,tc,0,matdim) 
    ACF = ACF +2*axialACF2mV2(taxis,e0,Vnarray,tc,1,matdim)
    ACF = ACF + 2*axialACF2mV2(taxis,e0,Vnarray,tc,2,matdim)
    return ACF

def axialte2mV2(e0,Vnarray,tc,mmag,Mdim=10):
    
    dVarray = np.array([Vnarray[1],3*Vnarray[2]])
    
    D = 1/(6*tc)
    
    matdim = Mdim + 1
    
    Pbararray = PbarRecurAll(e0,Vnarray,matdim+2)
    
    Y2Ypbar = np.zeros(matdim)
    for p in range(0,matdim):
        if p == 0 and mmag == 0:
            Y2Ypbar[p] = (-1)**mmag*(2*(p+2)+1)*wigner3j(p,p+2,mmag)*Pbararray[p+2]
        elif p == 1 and mmag <= 1:
            Y2Ypbar[p] = (2*p+1)*wigner3j(p,p,mmag)*Pbararray[p] + (2*(p+2)+1)*wigner3j(p,p+2,mmag)*Pbararray[p+2]
            Y2Ypbar[p] = (-1)**mmag*Y2Ypbar[p]
        elif p >= 2:
            Y2Ypbar[p] = (2*(p-2)+1)*wigner3j(p,p-2,mmag)*Pbararray[p-2] + (2*p+1)*wigner3j(p,p,
                        mmag)*Pbararray[p] + (2*(p+2)+1)*wigner3j(p,p+2,mmag)*Pbararray[p+2]
            Y2Ypbar[p] = (-1)**mmag*Y2Ypbar[p]
        if mmag == 0:
            Y2Ypbar[p] = Y2Ypbar[p] - Pbararray[2]*Pbararray[p]
    Y2Ypbar = Y2Ypbar[1:]
    
    rmat = np.zeros([matdim,matdim])

    #construct rate matrix
    for l in range(0,matdim):
        cll = -3*e0*dVarray[1]*(l*(l+1)-3*mmag**2)/((2*l+3)*(2*l+1))
        rmat[l,l] = l*(l+1) - cll
        
        if l > 0 and l >= mmag:
            rmat[l,l-1] = (e0*dVarray[0]/(2*l+1))*(l+1)*np.sqrt(l**2-mmag**2)
        if l < matdim - 1 and l+1 >= mmag:
            rmat[l,l+1] = -(e0*dVarray[0]/(2*l+1))*l*np.sqrt((l+1)**2-mmag**2)
        if l > 1 and l-1 >= mmag:
            rmat[l,l-2] = (3*e0*dVarray[1]/((2*l+1)*(2*l-1)))*(l+1)*np.sqrt(l**2-mmag**2)*np.sqrt((l-1)**2-mmag**2)
        if l < matdim - 2 and l+1 >= mmag:
            rmat[l,l+2] = -(3*e0*dVarray[1]/((2*l+1)*(2*l+3)))*l*np.sqrt((l+1)**2-mmag**2)*np.sqrt((l+2)**2-mmag**2)
    rmat = rmat[1:,1:]
    
    Minv = np.linalg.inv(rmat)
    
    taue = Minv[1,:]@Y2Ypbar/D
    
    return np.real(taue)

def axialteV2(e0,Vnarray,tc,Mdim=10):
    S2 = axialS2V2(e0,Vnarray)
    taue = axialte2mV2(e0,Vnarray,tc,0,Mdim) 
    taue = taue + 2*axialte2mV2(e0,Vnarray,tc,1,Mdim) 
    taue = taue +  2*axialte2mV2(e0,Vnarray,tc,2,Mdim)
    taue = taue/(1-S2)
    return taue

    return taue

def axialS2V2(e0,Vnarray):
    tmp = Pbarnum(e0,Vnarray,3)
    return tmp[2]**2

def avethetaV2(eps0,Vnarray):
    xarray = np.linspace(0,np.pi,1000)
    P0 = 1
    P1 = np.cos(xarray)
    P2 = (3*np.cos(xarray)**2-1)/2
    V = Vnarray[0]*P0 + Vnarray[1]*P1 + Vnarray[2]*P2
    
    avet = np.zeros(len(eps0))
    
    #calculate probability distribution
    for i,e0 in enumerate(eps0):
        tmp = np.exp(-e0*V)*np.sin(xarray)
        Z = (np.sum(2*tmp)-tmp[0]-tmp[-1])*np.pi/(2*len(xarray))
        Pdist = tmp/Z
    
        tmp = xarray*Pdist*np.sin(xarray)
        avet[i] = (np.sum(2*tmp)-tmp[0]-tmp[-1])*np.pi/(2*len(xarray))
    
    return avet

In [ ]:
fig = plt.figure(figsize=(4,8))

taxis = np.linspace(0,5,500)

mdim = 20
e0 = 8
tc = 1
S2 = axialS2(e0)
ACF10 = axialACF(taxis,e0,tc,mdim)
taue = axialte(e0,tc,mdim)
xAxial = avetheta(e0)*180/np.pi

# Diffusion-in-a-cone
thetarray = np.linspace(0.01,179.0,10000)
xdcone = avethetadcone(thetarray*np.pi/180)*180/np.pi
pfit = np.polynomial.polynomial.Polynomial.fit(xdcone,thetarray,deg=10)
cfit = pfit.convert()
cfit = cfit.coef
thetacone0 = np.polynomial.polynomial.polyval(xAxial,cfit)
dconeS2 = DiffConeS2(thetacone0)
dconete = DiffConete(thetacone0,tc)

ACFcone = DiffConeACF(taxis,thetacone0,tc)

print("S2:",S2,dconeS2,"tau_e",taue,dconete)
print("Ave theta",xAxial,"Wall",thetacone0)

plt.subplot(211,aspect=360/e0)
tarray = np.linspace(-180,180,500)
plt.plot(tarray,axialPotential(tarray,e0),c='black')
plt.axvline(thetacone0,c=Blue,ls=":")
plt.axvline(-thetacone0,c=Blue,ls=":")
plt.xlim(-180,180)
plt.ylim(0,e0)
plt.xlabel(r'$\theta$',fontsize=14)
plt.ylabel(r'$V(\theta)/(k_BT)$',fontsize=14)
plt.text(180*0.7,0.05*8,"(a)",fontsize=12)

plt.subplot(212,aspect=5)
#plt.plot(taxis,ACFV1,c=Green,ls="--")
plt.plot(taxis,ACFcone,c=Blue,ls="-.")
plt.plot(taxis,S2 + (1-S2)*np.exp(-taxis/taue),c=ReddishPurple,ls='--')
plt.axhline(axialS2(e0),c='black',ls=':')
plt.plot(taxis,ACF10,c='black')

plt.xlim(0,5)
plt.ylim(0,1)
plt.ylabel(r'$C(\tau)$',fontsize=14)
plt.xlabel(r'$D\tau$',fontsize=14)
plt.text(0.95*4.5,0.9,"(b)",fontsize=12)

plt.tight_layout()

plt.savefig("Figure6_DiffinCone_ACF.png",dpi=300,pad_inches=0)
plt.savefig("Figure6_DiffinCone_ACF.eps",pad_inches=0)

plt.show()

In [ ]:
fig = plt.figure(figsize=(4,12))

tc = 1
stdarray = np.linspace(2.55,250,2000)
epsarray = 2*(stdarray*np.pi/180)**-2
print(min(epsarray),max(epsarray))
x = avetheta(epsarray)*180/np.pi
daxialS2 = np.zeros(len(epsarray))
daxialte = np.zeros(len(epsarray))
for i,val in enumerate(epsarray):
    daxialS2[i] = axialS2(val)
    daxialte[i] = axialte(val,tc,Mdim=10)

thetarray = np.linspace(0.01,179.0,2000)
dconeS2 = DiffConeS2(thetarray)
dconete = DiffConete(thetarray,tc)
xdcone = avethetadcone(thetarray*np.pi/180)*180/np.pi

plt.subplot(311,aspect=90)
plt.plot(xdcone,dconeS2,c=ReddishPurple,ls='--')
plt.plot(x,daxialS2,c='black')
plt.xlim(0,90)
plt.ylim(0,1)
plt.xlabel(r'$\left< \theta \right>$',fontsize=14)
plt.ylabel(r'$S^2$',fontsize=14)
plt.text(80,0.9,"(a)",fontsize=12)

plt.subplot(312,aspect=90/1.2)
#plt.plot(xdcone,dconete,c=ReddishPurple)
plt.plot(x,daxialte,c='black')
plt.xlim(0,90)
plt.ylim(0,1.2)
plt.xlabel(r'$\left< \theta \right>$',fontsize=14)
plt.ylabel(r'$D\tau_e$',fontsize=14)
plt.text(80,0.9*1.2,"(b)",fontsize=12)

plt.subplot(313,aspect=1/1.2)
#plt.plot(dconeS2,dconete,c=ReddishPurple)
plt.plot(daxialS2,daxialte,c='black')
plt.xlim(0,1)
plt.ylim(0,1.2)
plt.xlabel(r'$S^2$',fontsize=14)
plt.ylabel(r'$D\tau_e$',fontsize=14)
plt.text(0.9,0.9*1.2,"(c)",fontsize=12)

plt.savefig("Figure7_DiffinCone_S2.png",dpi=300,pad_inches=0)
plt.savefig("Figure7_DiffinCone_S2.eps",pad_inches=0)

plt.show()

In [ ]:
fig = plt.figure(figsize=(4,8))

taxis = np.linspace(0,10,500)

mdim = 30
e0 = 8
tc = 1

ACFV1 =axialACF(taxis,e0,tc,mdim)

Vnarray2 = np.array([2/3.0,-1/2.0,-1/6.0])
#Vnarray2 = np.array([1/2.0,0,-1/2.0])

ACFV2 =axialACFV2(taxis,e0,Vnarray2,tc,mdim)
S2 = axialS2V2(e0,Vnarray2)
taue = axialteV2(e0,Vnarray2,tc,mdim)

S20 = axialS2(e0)
taue0 = axialte(e0,tc,mdim)

# Diffusion-in-a-cone
x0 = (1/2)*(-1+np.sqrt(1 + 8*np.sqrt(S2)))
thetacone0 = np.arccos(x0)*180/np.pi
dconeS2 = DiffConeS2(thetacone0)
dconete = DiffConete(thetacone0,tc)
ACFcone = DiffConeACF(taxis,thetacone0,tc)
aveTheta = avethetadcone(thetacone0*np.pi/180)*180/np.pi

print("Diffusion wall:",thetacone0,"Ave theta",aveTheta)
print("S2cone:",S2,dconeS2,"tau_e",taue,dconete)
print("S2loworder",S20,"tau_e",taue0)

plt.subplot(211,aspect=360/e0)
tarray = np.linspace(-180,180,500)
plt.plot(tarray,axialPotential(tarray,e0),c=Orange,ls=":",lw=2.5)
plt.plot(tarray,axialPotentialV2(tarray,e0,Vnarray2),c="black",ls="-")
plt.axvline(thetacone0,c=Blue,ls=":")
plt.axvline(-thetacone0,c=Blue,ls=":")
plt.xlim(-180,180)
plt.ylim(0,e0)
plt.xlabel(r'$\theta$',fontsize=14)
plt.ylabel(r'$V(\theta)/(k_BT)$',fontsize=14)
plt.text(0.7*180,8*0.05,"(a)",fontsize=12)

plt.subplot(212,aspect=5)
plt.plot(taxis,ACFV1,c=Orange,ls=":",lw=2.5)
plt.plot(taxis,ACFcone,c=Blue,ls="-.")
plt.plot(taxis,ACFV2,c='black',ls="-")
plt.plot(taxis,S2 + (1-S2)*np.exp(-taxis/taue),c=ReddishPurple,ls="--")
plt.axhline(S2,c='black',ls=':')

plt.xlim(0,5)
plt.ylim(0,1)
plt.ylabel(r'$C(\tau)$',fontsize=14)
plt.xlabel(r'$D\tau$',fontsize=14)
plt.text(0.95*4.5,0.9,"(b)",fontsize=12)

plt.tight_layout()

plt.savefig("Figure8_DiffinConeV2_ACF.png",dpi=300,pad_inches=0)
plt.savefig("Figure8_DiffinConeV2_ACF.eps",pad_inches=0)

plt.show()

In [ ]:
fig = plt.figure(figsize=(4,12))

Vnarray = np.array([2/3.0,-1/2.0,-1/6.0])
#Vnarray = np.array([1/2.0,0,-1/2.0])

tc = 1
stdarray = np.linspace(4.35,250,2000)
epsarray = 2*(stdarray*np.pi/180)**-2
print(min(epsarray),max(epsarray))

x = avethetaV2(epsarray,Vnarray)*180/np.pi
daxialS2 = np.zeros(len(epsarray))
daxialte = np.zeros(len(epsarray))
for i,val in enumerate(epsarray):
    daxialS2[i] = axialS2V2(val,Vnarray2)
    daxialte[i] = axialteV2(val,Vnarray2,tc,Mdim=60)

thetarray = np.linspace(0.01,179.0,2000)
dconeS2 = DiffConeS2(thetarray)
dconete = DiffConete(thetarray,tc)
xdcone = avethetadcone(thetarray*np.pi/180)*180/np.pi

plt.subplot(311,aspect=70)
plt.plot(xdcone,dconeS2,c=ReddishPurple,ls='--')
plt.plot(x,daxialS2,c='black')
plt.xlim(0,70)
plt.ylim(0,1)
plt.xlabel(r'$\left< \theta \right>$',fontsize=14)
plt.ylabel(r'$S^2$',fontsize=14)
plt.text(60,0.9*1.,"(a)",fontsize=12)

plt.subplot(312,aspect=70/1.1)
#plt.plot(xdcone,dconete,c=ReddishPurple)
plt.plot(x,daxialte,c='black')
plt.xlim(0,70)
plt.ylim(0,1.1)
plt.xlabel(r'$\left< \theta \right>$',fontsize=14)
plt.ylabel(r'$D\tau_e$',fontsize=14)
plt.text(5,0.9*1.1,"(b)",fontsize=12)

plt.subplot(313,aspect=1/1.1)
#plt.plot(dconeS2,dconete,c=ReddishPurple)
plt.plot(daxialS2,daxialte,c='black')
plt.xlim(0,1)
plt.ylim(0,1.1)
plt.xlabel(r'$S^2$',fontsize=14)
plt.ylabel(r'$D\tau_e$',fontsize=14)
plt.text(0.9,0.9*1.1,"(c)",fontsize=12)

plt.savefig("Figure9_DiffinConeV2_S2.png",dpi=300,pad_inches=0)
plt.savefig("Figure9_DiffinConeV2_S2.eps",pad_inches=0)

plt.show()